<a href="https://colab.research.google.com/github/Hu-Hao/quant-learning/blob/main/examples/colab_example_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantitative Trading Framework - Complete Example

This notebook demonstrates:
- **Our Custom Framework**: Complete backtesting with real market data
- **VectorBT Cross-Validation**: Compare results with industry-standard library
- **Side-by-Side Analysis**: Verify strategy implementation accuracy

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install yfinance pandas numpy matplotlib seaborn vectorbt

# Clone the quantitative trading framework from GitHub
!git clone https://github.com/Hu-Hao/quant-learning.git

# Add the project to Python path
import sys
sys.path.append('/content/quant-learning')

print("✅ Installation completed successfully!")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our framework
from quant_trading.strategies.moving_average import MovingAverageStrategy
from quant_trading.backtesting.engine import BacktestEngine

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully!")

## 2. Data Preparation

In [ ]:
# Fetch Apple stock data
symbol = "AAPL"
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # 1 year of data

print(f"Fetching {symbol} data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")

ticker = yf.Ticker(symbol)
data = ticker.history(start=start_date, end=end_date)
data.columns = [col.lower() for col in data.columns]  # Lowercase for compatibility

print(f"✅ Fetched {len(data)} days of data")
print(f"Price range: ${data['low'].min():.2f} - ${data['high'].max():.2f}")
print(f"Total return: {(data['close'].iloc[-1]/data['close'].iloc[0] - 1)*100:.1f}%")

In [ ]:
# Visualize the data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Price chart
ax1.plot(data.index, data['close'], label='Close Price', linewidth=1.5, color='blue')
ax1.plot(data.index, data['close'].rolling(20).mean(), label='20-day MA', linewidth=1, color='orange')
ax1.plot(data.index, data['close'].rolling(50).mean(), label='50-day MA', linewidth=1, color='red')
ax1.set_title(f'{symbol} Stock Price with Moving Averages')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Volume chart
ax2.bar(data.index, data['volume'], alpha=0.7, color='green')
ax2.set_title(f'{symbol} Trading Volume')
ax2.set_ylabel('Volume')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Strategy Setup

In [ ]:
# Create Moving Average strategy
strategy = MovingAverageStrategy(
    short_window=10,
    long_window=30,
    quantity=100
)

# Backtest parameters
initial_capital = 100000  # $100,000
commission = 0.001        # 0.1%
slippage = 0.001         # 0.1%

print(f"📊 Strategy: {strategy.get_name()}")
print(f"Parameters: {strategy.get_parameters()}")
print(f"Initial capital: ${initial_capital:,}")
print(f"Commission: {commission:.1%}")

## 4. Our Framework Backtest

In [ ]:
# Run backtest with our framework
print("🚀 Running backtest with our framework...")

engine = BacktestEngine(
    initial_capital=initial_capital,
    commission=commission,
    slippage=slippage,
    allow_short_selling=False  # Beginner-friendly
)

engine.run_backtest(data, strategy)
performance = engine.get_performance_summary()

# Display results
final_value = engine.portfolio_values[-1]
total_return = (final_value / initial_capital - 1) * 100
buy_hold_return = (data['close'].iloc[-1] / data['close'].iloc[0] - 1) * 100

print(f"\n📊 OUR FRAMEWORK RESULTS:")
print(f"   Final Value: ${final_value:,.2f}")
print(f"   Total Return: {total_return:.2f}%")
print(f"   Buy & Hold: {buy_hold_return:.2f}%")
print(f"   Outperformance: {total_return - buy_hold_return:+.2f}%")
print(f"   Total Trades: {len(engine.trades)}")
print(f"   Sharpe Ratio: {performance.get('sharpe_ratio', 0):.3f}")
print(f"   Max Drawdown: {performance.get('max_drawdown', 0)*100:.1f}%")

In [ ]:
# Create comprehensive visualization
fig = engine.plot_backtest_results(
    strategy_name="Moving Average Strategy",
    symbol=symbol,
    show_plot=True
)

if fig:
    print("✅ Comprehensive backtest visualization created!")
    print("\n🔍 Chart Interpretation:")
    print("   • Green ▲ = Buy signals")
    print("   • Red ▼ = Sell signals")
    print("   • Blue line = Strategy performance")
    print("   • Gray line = Buy & Hold benchmark")
else:
    print("⚠️ Visualization not available")

## 5. VectorBT Cross-Validation

Now let's validate our results by comparing with VectorBT, a professional backtesting library.

In [ ]:
# VectorBT Direct Integration - Learn VectorBT APIs
print("🔄 VECTORBT DIRECT USAGE")
print("=" * 40)

try:
    import vectorbt as vbt
    
    # Step 1: Generate signals using our strategy
    print("Step 1: Generating signals using our strategy...")
    entries = pd.Series(False, index=data.index)
    exits = pd.Series(False, index=data.index)
    
    # Use the same iteration pattern as our BacktestEngine
    for idx, _ in data.iterrows():
        partial_data = data.loc[:idx]
        signals = strategy.get_signals(partial_data)
        
        for signal in signals:
            if signal.action.value == 'buy':
                entries.at[idx] = True
            elif signal.action.value == 'sell':
                exits.at[idx] = True
    
    print(f"   Generated {entries.sum()} buy signals and {exits.sum()} sell signals")
    
    # Step 2: Create VectorBT Portfolio directly
    print("\nStep 2: Creating VectorBT Portfolio...")
    
    # VectorBT Portfolio.from_signals() - core VectorBT functionality
    vbt_portfolio = vbt.Portfolio.from_signals(
        close=data['close'],
        entries=entries,
        exits=exits,
        init_cash=initial_capital,
        fees=commission,
        freq='D'  # Daily frequency
    )
    
    print("   ✅ VectorBT Portfolio created successfully!")
    
    # Step 3: Extract VectorBT performance metrics directly
    print("\nStep 3: Extracting VectorBT performance metrics...")
    
    # Direct VectorBT stats access
    vbt_stats = vbt_portfolio.stats()
    vbt_total_return = vbt_stats['Total Return [%]'] / 100
    vbt_sharpe = vbt_stats['Sharpe Ratio']
    vbt_max_dd = vbt_stats['Max Drawdown [%]'] / 100
    vbt_total_trades = vbt_stats['# Trades']
    
    print(f"   VectorBT Total Return: {vbt_total_return*100:.2f}%")
    print(f"   VectorBT Sharpe Ratio: {vbt_sharpe:.3f}")
    print(f"   VectorBT Max Drawdown: {vbt_max_dd*100:.1f}%")
    print(f"   VectorBT Total Trades: {vbt_total_trades}")
    
    # Step 4: Compare with our framework results
    print(f"\n📊 DIRECT COMPARISON:")
    print(f"   Our Framework: {total_return:.2f}%")
    print(f"   VectorBT:      {vbt_total_return*100:.2f}%")
    print(f"   Difference:    {abs(total_return - vbt_total_return*100):.2f}%")
    
    # Calculate correlation of portfolio values
    our_portfolio_series = pd.Series(engine.portfolio_values, index=data.index[:len(engine.portfolio_values)])
    vbt_portfolio_series = vbt_portfolio.value()
    
    # Align series for correlation calculation
    common_index = our_portfolio_series.index.intersection(vbt_portfolio_series.index)
    correlation = our_portfolio_series.loc[common_index].corr(vbt_portfolio_series.loc[common_index])
    
    print(f"   Correlation:   {correlation:.3f}")
    
    if correlation > 0.95:
        print("   ✅ VALIDATION SUCCESSFUL - High correlation!")
    else:
        print("   ⚠️ Some differences detected - review implementation")
    
    # Step 5: VectorBT's powerful analytics
    print(f"\n💡 VectorBT Advanced Analytics:")
    print(f"   Win Rate: {vbt_stats['Win Rate [%]']:.1f}%")
    print(f"   Profit Factor: {vbt_stats['Profit Factor']:.2f}")
    print(f"   Avg Trade: {vbt_stats['Avg Trade [%]']:.2f}%")
    print(f"   Best Trade: {vbt_stats['Best Trade [%]']:.2f}%")
    print(f"   Worst Trade: {vbt_stats['Worst Trade [%]']:.2f}%")
    
    # Store VectorBT objects for plotting
    vbt_available = True
    
except ImportError:
    print("⚠️ VectorBT not installed. Install with: pip install vectorbt")
    vbt_available = False
except Exception as e:
    print(f"⚠️ VectorBT error: {e}")
    vbt_available = False

In [ ]:
# Direct VectorBT Visualization - Learn VectorBT Plotting APIs
print("📊 Creating Direct VectorBT Visualizations...")

if vbt_available:
    # Create comprehensive comparison using VectorBT's native plotting
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Subplot 1: Price with signals using VectorBT
    ax1 = axes[0, 0]
    
    # Plot price and moving averages
    ax1.plot(data.index, data['close'], label='Close Price', linewidth=1.5, color='blue')
    ax1.plot(data.index, data['close'].rolling(strategy.params['short_window']).mean(), 
             label=f"MA{strategy.params['short_window']}", linewidth=1, color='orange')
    ax1.plot(data.index, data['close'].rolling(strategy.params['long_window']).mean(), 
             label=f"MA{strategy.params['long_window']}", linewidth=1, color='red')
    
    # Add VectorBT signals directly
    entry_prices = data['close'][entries]
    exit_prices = data['close'][exits]
    
    ax1.scatter(entry_prices.index, entry_prices.values, 
               marker='^', color='green', s=100, label='VectorBT Buy', zorder=5)
    ax1.scatter(exit_prices.index, exit_prices.values, 
               marker='v', color='red', s=100, label='VectorBT Sell', zorder=5)
    
    ax1.set_title('Price Chart with VectorBT Signals')
    ax1.set_ylabel('Price ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Subplot 2: Portfolio performance comparison
    ax2 = axes[0, 1]
    
    # Our framework portfolio
    our_portfolio_series = pd.Series(engine.portfolio_values, index=data.index[:len(engine.portfolio_values)])
    ax2.plot(our_portfolio_series.index, our_portfolio_series.values, 
             label='Our Framework', linewidth=2, color='blue')
    
    # VectorBT portfolio using .value()
    vbt_portfolio_values = vbt_portfolio.value()
    ax2.plot(vbt_portfolio_values.index, vbt_portfolio_values.values, 
             label='VectorBT', linewidth=2, color='red', linestyle='--')
    
    # Buy & Hold benchmark
    buy_hold_values = initial_capital * (data['close'] / data['close'].iloc[0])
    ax2.plot(data.index, buy_hold_values, 
             label='Buy & Hold', linewidth=1, color='gray', linestyle=':')
    
    ax2.set_title('Portfolio Performance Comparison')
    ax2.set_ylabel('Portfolio Value ($)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Subplot 3: Performance metrics comparison
    ax3 = axes[1, 0]
    
    # Compare key metrics
    metrics = ['Total Return (%)', 'Sharpe Ratio', 'Max Drawdown (%)']
    our_values = [total_return, performance.get('sharpe_ratio', 0), performance.get('max_drawdown', 0)*100]
    vbt_values = [vbt_total_return*100, vbt_sharpe, vbt_max_dd*100]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    bars1 = ax3.bar(x - width/2, our_values, width, label='Our Framework', color='blue', alpha=0.7)
    bars2 = ax3.bar(x + width/2, vbt_values, width, label='VectorBT', color='red', alpha=0.7)
    
    ax3.set_title('Performance Metrics Comparison')
    ax3.set_ylabel('Value')
    ax3.set_xticks(x)
    ax3.set_xticklabels(metrics, rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}', ha='center', va='bottom', fontsize=8)
    for bar in bars2:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}', ha='center', va='bottom', fontsize=8)
    
    # Subplot 4: VectorBT specific analytics
    ax4 = axes[1, 1]
    
    # VectorBT trade analysis using .trades accessor
    trades = vbt_portfolio.trades
    if len(trades.records) > 0:
        # Plot trade returns distribution
        trade_returns = trades.returns.values
        ax4.hist(trade_returns, bins=20, alpha=0.7, color='purple', edgecolor='black')
        ax4.axvline(trade_returns.mean(), color='red', linestyle='--', 
                   label=f'Mean: {trade_returns.mean():.2%}')
        ax4.set_title('VectorBT Trade Returns Distribution')
        ax4.set_xlabel('Trade Return')
        ax4.set_ylabel('Frequency')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
    else:
        ax4.text(0.5, 0.5, 'No Trades to Analyze', ha='center', va='center', 
                transform=ax4.transAxes, fontsize=12)
        ax4.set_title('VectorBT Trade Analysis')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Direct VectorBT visualization created!")
    print("\n🔍 What You're Learning from VectorBT:")
    print("   📈 Portfolio.from_signals() - Core backtesting method")
    print("   📊 .stats() - Comprehensive performance metrics")
    print("   📉 .value() - Portfolio value time series")
    print("   🔄 .trades - Individual trade analysis")
    print("   ⚡ Vectorized operations for speed")
    
    # Advanced VectorBT features demonstration
    print(f"\n🚀 Advanced VectorBT Features Used:")
    print(f"   • Portfolio class for complete backtesting")
    print(f"   • Automated fee/commission handling")
    print(f"   • Built-in performance statistics")
    print(f"   • Trade-level analytics")
    print(f"   • Efficient vectorized operations")
    
    # Show VectorBT's powerful .stats() output
    print(f"\n📋 Complete VectorBT Stats (learn the API):")
    print(vbt_stats.to_string())
    
else:
    print("⚠️ VectorBT not available - install to see direct API usage")
    print("📚 Learning points (when VectorBT is installed):")
    print("   • vbt.Portfolio.from_signals() for backtesting")
    print("   • .stats() for performance metrics")
    print("   • .value() for portfolio time series")
    print("   • .trades for trade analysis")
    print("   • Native plotting capabilities")

## 6. Summary and Insights

In [ ]:
print("🎯 COMPLETE ANALYSIS SUMMARY")
print("=" * 50)

print(f"\n📊 Strategy Performance:")
print(f"   Symbol: {symbol}")
print(f"   Period: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
print(f"   Strategy: Moving Average ({strategy.params['short_window']}/{strategy.params['long_window']})")
print(f"   Total Return: {total_return:.2f}%")
print(f"   Buy & Hold: {buy_hold_return:.2f}%")
print(f"   Outperformance: {total_return - buy_hold_return:+.2f}%")

if 'comparison' in locals() and comparison['status']:
    print(f"\n🔄 Cross-Validation:")
    print(f"   Status: {comparison['status']}")
    print(f"   Framework Agreement: {comparison['correlation']:.3f}")
    if comparison['status'] == 'PASS':
        print("   ✅ Implementation validated successfully!")
    else:
        print("   ⚠️ Minor differences detected - review recommended")

print(f"\n💡 Key Learning Points:")
print(f"   ✅ Built complete backtesting framework from scratch")
print(f"   ✅ Tested with real market data (Yahoo Finance)")
print(f"   ✅ Cross-validated against industry standard (VectorBT)")
print(f"   ✅ Comprehensive visualization and analysis tools")
print(f"   ✅ Professional-grade performance metrics")

print(f"\n🚀 Next Steps:")
print(f"   • Try different strategy parameters")
print(f"   • Test other strategies (Momentum, Mean Reversion)")
print(f"   • Experiment with different assets")
print(f"   • Add risk management features")
print(f"   • Parameter optimization with VectorBT speed")

print(f"\n📚 Framework Benefits Demonstrated:")
print(f"   ✅ Beginner-friendly (no short selling)")
print(f"   ✅ Realistic execution costs")
print(f"   ✅ Professional visualization")
print(f"   ✅ Cross-validation capability")
print(f"   ✅ Easy to extend and customize")

## 🎉 Congratulations!

You've successfully:

1. **Built a Complete Trading System**: From data fetching to result visualization
2. **Implemented Professional Features**: Realistic costs, risk management, comprehensive metrics
3. **Validated Your Implementation**: Cross-checked against industry-standard VectorBT library
4. **Created Publication-Quality Visualizations**: Charts suitable for research and presentations

### 🔍 What the Cross-Validation Proves:
- Your strategy logic is implemented correctly
- Performance calculations are accurate
- Signal generation timing is precise
- Results can be trusted for decision-making

### 📈 Framework Advantages:
- **Custom Control**: Full control over execution logic
- **Educational Value**: Understand every component
- **Flexibility**: Easy to add new features
- **Validation**: Cross-check with professional tools

### ⚠️ Important Reminder:
This is for **educational purposes only**. Always:
- Test thoroughly before any real trading
- Start with paper trading
- Consult financial professionals
- Past performance ≠ future results